## Atelier pratique : attaques adversariales - 1

Implémentation guidée d’une attaque adversariale. <br>
Attaques :  **FGSM** et **PGD** <br>
Modèle cible : un petit modèle d’image fait à la main sur MNIST <br>
Framework : PyTorch <br>

<span style="color: orange;">Importer les dépendances de PyTorch.</span>

In [ ]:
from pathlib import Path

# ✏️ Votre code ici

In [ ]:
DATA_DIR = Path("../data")

<details>
<summary>💡 Indice</summary>
Utiliser `torch`, `torch.nn`, `torch.optim`, `torchvision`, `torchvision.transforms`.
</details>

<details>
<summary>💻 Voir la solution</summary>

👉 [Solution](./_solutions/tp1/1_imports.py)

</details>

<span style="color: orange;">Charger le dataset MNIST à partir de torchvision.datasets.MNIST.</span>

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.ToTensor()

# ✏️ Votre code ici

<details>
<summary>💡 Indice</summary>
Utiliser un DataLoader de `torch.utils.data`.
</details>

<details>
<summary>💻 Voir la solution</summary>

👉 [Solution](./_solutions/tp1/2_data_loader.py)

</details>

<span style="color: orange;">Définir un petit CNN destiné à classer les images de MNIST.</span>

In [ ]:
# ✏️ Votre code ici

<details>
<summary>💡 Indice</summary>
Une architecture Conv2d → ReLU → Linear suffit. <br>
Définir une classe héritant de nn.Module. <br>
Dans l'__init__, créer une couche Conv2d ainsi qu'une couche Linear, en suivant bien les tailles des objets manipulés. <br>
Dans le forward, appliquer ces étapes ainsi qu'un flatten placé au bon endroit (exemple de syntaxe : x.view(x.size(0), -1)).

</details>

<details>
<summary>💻 Voir la solution</summary>

👉 [Solution](./_solutions/tp1/3_simple-cnn.py)

</details>

<span style="color: orange;">Instancier le modèle (et l'envoyer sur le device). Instancier également les outils (une fonction de perte et un optimizer classiques).</span>

In [ ]:
model = ....to(device) # ✏️ à compléter
criterion = ... # ✏️ à compléter
optimizer = ... # ✏️ à compléter

<details>
<summary>💡 Indice</summary>
Pour un problème de classification multiclasse, la fonction de perte standard est CrossEntropyLoss. <br>
Utiliser par exemple comme optimizer Adam avec un taux d'apprentissage de 0.001.

</details>

<details>
<summary>💻 Voir la solution</summary>

👉 [Solution](./_solutions/tp1/4_instanciation.py)

</details>

<span style="color: orange;">Effectuer un entraînement rapide.</span>

In [ ]:
n_epochs = 3
for epoch in range(n_epochs):
    for images, labels in ...: # ✏️ à compléter
        images, labels = images.to(device), labels.to(device)
        optimizer.... # ✏️ à compléter
        outputs = ... # ✏️ à compléter
        loss = # ✏️ à compléter
        ... # ✏️ à compléter

<details>
<summary>💡 Indice</summary>
Dans la boucle interne, on récupère deux éléments donnés par un itérable PyTorch généralement utilisé pour charger des mini‑batches. <br>
Avant le calcul du gradient, une opération essentielle doit être faite pour éviter l’accumulation des gradients d’une itération à l’autre (PyTorch les accumule par défaut). <br>
Faire une passe directe (forward). <br>
Un outil créé lors de l’initialisation de l’entraînement sert à comparer les prédictions aux labels, créant un tenseur scalaire. <br>
Backpropagation. <br>
Mise à jour des poids.
</details>

<details>
<summary>💻 Voir la solution</summary>

👉 [Solution](./_solutions/tp1/5_training.py)

</details>

<span style="color: orange;">Implémenter l'attaque FGSM.</span>

In [ ]:
def fgsm_attack(model, images, labels, ...):  # ✏️ à compléter
    images = images.clone().detach().requires_grad_(True)
    
    outputs = ... # ✏️ à compléter
    loss = ... # ✏️ à compléter
    
    model.zero_grad(set_to_none=True)
    loss.... # ✏️ à compléter
    
    adv_images = torch.clamp(..., ..., ...) # ✏️ à compléter
    return adv_images

<details>
<summary>💡 Indice</summary>
FGSM : epsilon * sign(gradient) <br>
Les calculs d'output et de loss sont-ils différents de ceux de l'entraînement ? <br>
Où rajouter la perturbation ? <br>
torch.clamp(image, min, max) sert à "forcer" les valeurs d’un tenseur dans un intervalle donné. L’ajout de perturbations peut faire dépasser les bornes. Quelles sont les bornes dans notre cas ? Ici nous sommes en convention float.
</details>

<details>
<summary>💻 Voir la solution</summary>

👉 [Solution](./_solutions/tp1/6_fgsm.py)

</details>

<span style="color: orange;">Implémenter l'attaque PGD.</span>

In [ ]:
def pgd_attack(model, images, labels, epsilon, alpha, iters):

    ori_images = images.clone().detach()
    # model.eval()  # optionnel, cf remarque

    for _ in range(iters):
        images = images.clone().detach().requires_grad_(True)

        with torch.enable_grad():
            outputs = ... # ✏️ à compléter
            loss = ... # ✏️ à compléter

        # ✏️ à compléter

    return images.detach()


<details>
<summary>💡 Indice</summary>
Repartir de la syntaxe de FGSM et gérer la boucle d'itérations supplémentaire.
</details>

<details>
<summary>💻 Voir la solution</summary>

👉 [Solution](./_solutions/tp1/7_pgd.py)

</details>

Selon votre implémentation, il peut y avoir quelques différences avec l'attaque réelle PGD. <br>
La solution présentée présente quelques possibilités d'améliorations pour tendre vers un PGD réel :
- l'initialisation se fait souvent (de manière aléatoire) dans la boule L<sub>∞</sub>(ϵ) au départ (x<sub>0</sub> = x + uniform(-ε, ε)),
- PGD peut être généralisé à d’autres normes (classiquement L<sub>2</sub> et L<sub>1</sub>),
- alpha peut être exprimé en fonction de ϵ (classiquement alpha ≈ ε / iters).

Enfin, une remarque : dans les véritables attaques itératives, on utilise généralement model.eval() pour s’assurer que chaque itération de l’attaque (et non les itérations d’entraînement comme les batchs ou les epochs) voit exactement le même modèle. <br>
Pour notre SimpleCNN, cela ne change rien, mais avec un modèle au comportement stochastique (en particulier lorsqu’il contient des couches Dropout ou BatchNorm) cela devient utile. <br>
Sans model.eval(), chaque itération de PGD serait effectuée avec un modèle légèrement différent, ce qui rend les gradients incohérents d’une itération à l’autre et diminue l’efficacité de l’attaque.

https://github.com/ultralytics/yolov5/issues/12808

<span style="color: orange;">Question bonus si vous êtes en avance : implémenter l'attaque PGD en L<sub>2</sub> plutôt que L<sub>∞</sub>.</span>

In [ ]:
# ✏️ Votre code ici

<span style="color: orange;">Tester et comparer les attaques FGSM et PGD.</span>

In [ ]:
# ✏️ Votre code ici

<details>
<summary>💡 Indice</summary>
Choisir une valeur de ε.
Itérer sur le jeu de test (le Loader associé).
Générer les attaques adverses sur les images de test.
Réaliser la classification à l'aide du modèle attaqué.
Calculer la précision associée.
</details>

<details>
<summary>💻 Voir la solution</summary>

👉 [Solution](./_solutions/tp1/8_comparaison.py)

</details>

Une première version simplifiée de l'attaque **Carlini & Wagner** a été implémentée et est fournie dans le bloc suivant. <span style="color: orange;">Est-elle performante ?</span>

In [ ]:
import torch
import torch.nn.functional as F

def cw_l22_attack(model, images, labels, c=1e-3, kappa=0, lr=0.01, iters=50):
    model.eval()
    batch_size = images.size(0)

    # tanh-space transform
    imgs_tanh = torch.atanh(images * 2 - 1)
    w = imgs_tanh.detach().clone().requires_grad_(True)

    optimizer = torch.optim.Adam([w], lr=lr)

    for _ in range(iters):
        optimizer.zero_grad()
        
        # get adversarial image
        adv = torch.tanh(w)
        adv = (adv + 1) / 2  # back to [0,1]

        # compute logits
        logits = model(adv)

        # correct class and max other class logits
        true_logit = logits.gather(1, labels.unsqueeze(1)).squeeze(1)
        max_other_logit = torch.max(
            logits.masked_fill(
                torch.nn.functional.one_hot(labels, logits.size(1)).bool(), 
                -1e9
            ), dim=1
        )[0]

        # CW objective f(x)
        f = torch.clamp(max_other_logit - true_logit + kappa, min=0)

        # L2^2 loss
        l2 = torch.sum((adv - images) ** 2, dim=[1,2,3])

        loss = l2 + c * f
        loss = loss.mean()

        loss.backward()
        optimizer.step()

    adv = torch.tanh(w)
    return (adv + 1) / 2

In [40]:
# ✏️ Votre code ici

<details>
<summary>💡 Indice</summary>
Lui faire subir le même sort qu'aux deux attaques précédentes et comparer les performances.
</details>

<details>
<summary>💻 Voir la solution</summary>

👉 [Solution](./_solutions/tp1/9_cw_simplifie.py)

</details>

L'implémentation nous paraît peu efficace. Nous allons utiliser une librairie dédiée pour effectuer une vraie attaque CW.

<span style="color: orange;">Installer la librairie **Torchattacks** :</span>

```bash
pip install torchattacks



<span style="color: orange;">Grâce à Torchattacks, appliquer l'attaque CW. Evaluer son efficacité et la comparer à celle de notre CW manuel ainsi qu'à celle des autres attaques.</span>

In [ ]:
# ✏️ Votre code ici

<details>
<summary>💡 Indice</summary>
Utiliser la classe `torchattacks.CW`.


Exemple d’utilisation :
```python
atk = torchattacks.CW(model, c=1e-3, lr=0.01, steps=50)
adv = atk(images, labels)



<details>
<summary>💻 Voir la solution</summary>

👉 [Solution](./_solutions/tp1/10_torchattacks_cw.py)

</details>

<span style="color: orange;">Torchattacks est-elle une librairie sérieuse ? Appliquer son implémentation de PGD.</span>

In [ ]:
# ✏️ Votre code ici

<details>
<summary>💡 Indice</summary>
CW devrait vous donner une idée de la classe à utiliser pour appliquer PGD.



<details>
<summary>💻 Voir la solution</summary>

👉 [Solution](./_solutions/tp1/11_torchattacks_pgd.py)

</details>

<span style="color: orange;">Donner une conclusion possible à ce TP.</span>

<details>
<summary>💻 Voir la solution</summary>

👉 [Solution](./_solutions/tp1/12_conclusion.txt)

</details>

<span style="color: orange;">Question bonus : ces attaques ont-elles besoin de connaître le vrai label de l'image ?</span>

<details>
<summary>💻 Voir la solution</summary>

👉 [Solution](./_solutions/tp1/13_connaissance-label.txt)

</details>